In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

# ✅ Chrome Options for Smooth Execution
#chrome_options = Options()
#chrome_options.add_argument("--disable-blink-features=AutomationControlled")
#chrome_options.add_argument("start-maximized")
#chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# ✅ Convert IMDb-style Duration to Minutes
def imdb_duration_to_minutes(duration):
    hours = re.search(r'(\d+)h', duration)
    minutes = re.search(r'(\d+)m', duration)
    
    total_minutes = (int(hours.group(1)) * 60 if hours else 0) + (int(minutes.group(1)) if minutes else 0)
    return total_minutes if total_minutes > 0 else "N/A"
# ✅ Start WebDriver
#driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()

# ✅ IMDb Movies Page (2024 Releases)
base_url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=action"
driver.get(base_url)
time.sleep(3)  # Let the page load

movie_count = 0

while True:
    # ✅ Wait for Movies to Load
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ipc-title-link-wrapper"))
        )
    except:
        print("❌ Timeout: IMDb page did not load.")
        break

    # ✅ Find All Movies on the Current Page
    movies = driver.find_elements(By.XPATH, "//li[contains(@class, 'ipc-metadata-list-summary-item')]")

    if not movies:
        print("❌ No movies found. IMDb may have changed its structure.")
        break

    for movie in movies:
        try:
            # 🎬 Movie Name
            name = movie.find_element(By.CLASS_NAME, "ipc-title-link-wrapper").text

            # ⭐ Ratings
            rating = movie.find_element(By.XPATH, "//span[contains(@class, 'ipc-rating-star--rating')]").text if movie.find_elements(By.XPATH, "//span[contains(@class, 'ipc-rating-star--rating')]") else "N/A"

            # 🗳 Voting Counts
            votes = movie.find_element(By.XPATH, "//span[contains(@class, 'ipc-rating-star--voteCount')]").text if movie.find_elements(By.XPATH, "//span[contains(@class, 'ipc-rating-star--voteCount')]") else "N/A"
            votes1 = votes.replace('(','').replace(')','')
        

            # ⏳ Duration
            duration_elements = movie.find_elements(By.XPATH, ".//div[contains(@class, 'dli-title-metadata')]")
            duration = duration_elements[0].text if duration_elements else "N/A"
            duration1 = duration.replace('2024','').replace('R', '').replace('PG', '').replace('-13', '')
            duration_in_minutes = imdb_duration_to_minutes(duration1)
            

            movie_count += 1
            print(f"{name}\t {rating}\t {votes1}\t {duration_in_minutes}m")

        except Exception as e:
            print(f"❌ Error extracting details for a movie: {e}")

    # ✅ Check for Next Page Button
    try:
        next_button = driver.find_element(By.XPATH, "//span[contains(text(), '50 more')]")
        next_button.click()
        time.sleep(10)  # Wait for next page to load
        print("page loaded")
    except:
        print("\n✅ No more pages. Scraping completed.")
        break

# ✅ Close browser
driver.quit()


1. Kraven the Hunter	 5.4	  44K	 127m
2. Gladiator II	 5.4	  44K	 148m
3. Sonic the Hedgehog 3	 5.4	  44K	 110m
4. Venom: The Last Dance	 5.4	  44K	 110m
5. Dune: Part Two	 5.4	  44K	 166m
6. Pushpa: The Rule - Part 2	 5.4	  44K	 201m
7. William Tell	 5.4	  44K	 133m
8. Elevation	 5.4	  44K	 91m
9. The Lord of the Rings: The War of the Rohirrim	 5.4	  44K	 134m
10. The Ministry of Ungentlemanly Warfare	 5.4	  44K	 122m
11. Furiosa: A Mad Max Saga	 5.4	  44K	 148m
12. Deadpool & Wolverine	 5.4	  44K	 128m
13. Borderlands	 5.4	  44K	 101m
14. Hellboy: The Crooked Man	 5.4	  44K	 99m
15. Twisters	 5.4	  44K	 122m
16. Civil War	 5.4	  44K	 109m
17. Old Guy	 5.4	  44K	 94m
18. The Count of Monte-Cristo	 5.4	  44K	 178m
19. The Beekeeper	 5.4	  44K	 105m
20. Carry-On	 5.4	  44K	 119m
21. Marco	 5.4	  44K	 145m
22. The Fall Guy	 5.4	  44K	 126m
23. Rebel Ridge	 5.4	  44K	 131m
24. Homestead	 5.4	  44K	 112m
25. The Crow	 5.4	  44K	 111m
26. Thelma	 5.4	  44K	 98m
27. Kingdom of the Planet of 

In [58]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import re
import csv
import os

# IMDb URL for 2024 movies
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=action"

# Initialize WebDriver
driver = webdriver.Chrome()  # Ensure you have ChromeDriver installed
driver.get(url)
sleep(5)  # Allow page to load

# ✅ Scroll Down Until All Movies Are Loaded
scroll_attempts = 10  # Adjust this if needed
for _ in range(scroll_attempts):
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    sleep(2)  # Wait for movies to load

# Scroll and click "50 more" until all movies are loaded
#while True:
#    try:
#        more_button = driver.find_element(By.XPATH, "//span[contains(text(), '50 more')]")
#        ActionChains(driver).move_to_element(more_button).click().perform()
#        sleep(1)
#        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)  # Wait for new movies to load
#    except:
#        break  # Stop when no more "50 more" button is found

# ✅ Convert IMDb-style Duration to Minutes
def imdb_duration_to_minutes(duration):
    hours = re.search(r'(\d+)h', duration)
    minutes = re.search(r'(\d+)m', duration)
    
    total_minutes = (int(hours.group(1)) * 60 if hours else 0) + (int(minutes.group(1)) if minutes else 0)
    return total_minutes if total_minutes > 0 else "N/A"

# ✅ Define CSV File Path (Same Folder as Jupyter Notebook)
csv_filename = os.path.join(os.getcwd(), "action.csv")

# ✅ Check if File Already Exists
file_exists = os.path.isfile(csv_filename)

# ✅ Open CSV file for writing
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    
    # ✅ Write header only if file is newly created
    if not file_exists:
        writer.writerow(["Movie Name", "Rating", "Votes", "Duration (minutes)"])    
    
    # Extract movie details
    movies = driver.find_elements(By.XPATH, "//li[contains(@class, 'ipc-metadata-list-summary-item')]")

    for movie in movies:
        try:
            # 🎬 Movie Name
            name = movie.find_element(By.CLASS_NAME, "ipc-title-link-wrapper").text

            # ⭐ Ratings
            rating = movie.find_element(By.XPATH, "//span[contains(@class, 'ipc-rating-star--rating')]").text if movie.find_elements(By.XPATH, "//span[contains(@class, 'ipc-rating-star--rating')]") else "N/A"

            # 🗳 Voting Counts
            votes = movie.find_element(By.XPATH, "//span[contains(@class, 'ipc-rating-star--voteCount')]").text if movie.find_elements(By.XPATH, "//span[contains(@class, 'ipc-rating-star--voteCount')]") else "N/A"
            votes1 = votes.replace('(','').replace(')','')
        

            # ⏳ Duration
            duration_elements = movie.find_elements(By.XPATH, ".//div[contains(@class, 'dli-title-metadata')]")
            duration = duration_elements[0].text if duration_elements else "N/A"
            duration1 = duration.replace('2024','').replace('R', '').replace('PG', '').replace('-13', '')
            duration_in_minutes = imdb_duration_to_minutes(duration1)
            
            movie_count += 1
             # ✅ Write Data to CSV
            writer.writerow([name, rating, votes1, duration_in_minutes])

            print(f"{name} | {rating} | {votes1} | {duration_in_minutes}m")

        except Exception as e:
            print(f"❌ Error extracting details for a movie: {e}")



# Close the browser
driver.quit()


1. Kraven the Hunter | 5.4 |  44K | 127m
2. Gladiator II | 5.4 |  44K | 148m
3. Sonic the Hedgehog 3 | 5.4 |  44K | 110m
4. Venom: The Last Dance | 5.4 |  44K | 110m
5. Dune: Part Two | 5.4 |  44K | 166m
6. Pushpa: The Rule - Part 2 | 5.4 |  44K | 201m
7. William Tell | 5.4 |  44K | 133m
8. Elevation | 5.4 |  44K | 91m
9. The Lord of the Rings: The War of the Rohirrim | 5.4 |  44K | 134m
10. The Ministry of Ungentlemanly Warfare | 5.4 |  44K | 122m
11. Furiosa: A Mad Max Saga | 5.4 |  44K | 148m
12. Deadpool & Wolverine | 5.4 |  44K | 128m
13. Borderlands | 5.4 |  44K | 101m
14. Hellboy: The Crooked Man | 5.4 |  44K | 99m
15. Twisters | 5.4 |  44K | 122m
16. Civil War | 5.4 |  44K | 109m
17. Old Guy | 5.4 |  44K | 94m
18. The Count of Monte-Cristo | 5.4 |  44K | 178m
19. The Beekeeper | 5.4 |  44K | 105m
20. Carry-On | 5.4 |  44K | 119m
21. Marco | 5.4 |  44K | 145m
22. The Fall Guy | 5.4 |  44K | 126m
23. Rebel Ridge | 5.4 |  44K | 131m
24. Homestead | 5.4 |  44K | 112m
25. The Crow |

In [ ]:
lkwsjdlksajdldwlkjdlksajfdlkjafldkajdlksajflksahflksahf